This notebook illustrates the issue with applying data classifications in Azure SQL DW and then using CTAS to load data. 
To use this demo, create an Azure SQL Data Warehouse and use the sample database. 
Run the first 5 code cells down through switching the tables with the RENAME command. See that data classifications are no longer on the dimension table. 
Then run the 6th code cell to clean up by deleting the extra tables. 
Run the first 4 code cells through creating the upsert table.
Run the cell to create the workaround stored procedure, and then run the subsequent cell to execute the stored procedure.

In [52]:
--Create some data classifications on DimCustomer
ADD SENSITIVITY CLASSIFICATION TO
    dbo.DimCustomer.EmailAddress, dbo.DimCustomer.Phone
    WITH ( LABEL='Confidential', INFORMATION_TYPE='Contact Info' )

Commands completed successfully.

Total execution time: 00:00:00.1084793

In [55]:
--Check that the data classifications now exist
SELECT
    sys.all_objects.name AS TableName, sys.all_columns.name As ColumnName,
    [Label], Label_ID, Information_Type, Information_Type_ID
FROM
          sys.sensitivity_classifications
LEFT OUTER JOIN sys.all_objects on sys.sensitivity_classifications.major_id = sys.all_objects.object_id
LEFT OUTER JOIN sys.all_columns on sys.sensitivity_classifications.major_id = sys.all_columns.object_id
                         and sys.sensitivity_classifications.minor_id = sys.all_columns.column_id

(2 rows affected)

Total execution time: 00:00:00.2399692

TableName,ColumnName,Label,Label_ID,Information_Type,Information_Type_ID
DimCustomer,EmailAddress,Confidential,NULL,Contact Info,NULL
DimCustomer,Phone,Confidential,NULL,Contact Info,NULL


In [49]:
--Create a staging table with some changed data 

CREATE TABLE dbo.Stg_Customer 
WITH
(   DISTRIBUTION = HASH([CustomerKey])
,   CLUSTERED COLUMNSTORE INDEX
)
AS
SELECT [CustomerKey]
      ,[GeographyKey]
      ,[CustomerAlternateKey]
      ,[Title] = Case Gender when 'F' then 'Miss' when 'M' then 'Mr' end
      ,[FirstName]
      ,[MiddleName]
      ,[LastName]
      ,[NameStyle]
      ,[BirthDate]
      ,[MaritalStatus]
      ,[Suffix]
      ,[Gender]
      ,[EmailAddress]
      ,[YearlyIncome]
      ,[TotalChildren]
      ,[NumberChildrenAtHome]
      ,[EnglishEducation]
      ,[SpanishEducation]
      ,[FrenchEducation]
      ,[EnglishOccupation]
      ,[SpanishOccupation]
      ,[FrenchOccupation]
      ,[HouseOwnerFlag]
      ,[NumberCarsOwned]
      ,[AddressLine1]
      ,[AddressLine2]
      ,[Phone]
      ,[DateFirstPurchase]
      ,[CommuteDistance]
  FROM [dbo].[DimCustomer]
  where customerkey < 15000

(4000 rows affected)

Total execution time: 00:00:02.1153974

In [50]:
--Create and populate the upsert table
CREATE TABLE dbo.[DimCustomer_upsert]
WITH
(   DISTRIBUTION = HASH([CustomerKey])
,   CLUSTERED COLUMNSTORE INDEX
)
AS
-- New rows and new versions of rows
SELECT [CustomerKey]
      ,[GeographyKey]
      ,[CustomerAlternateKey]
      ,[Title] 
      ,[FirstName]
      ,[MiddleName]
      ,[LastName]
      ,[NameStyle]
      ,[BirthDate]
      ,[MaritalStatus]
      ,[Suffix]
      ,[Gender]
      ,[EmailAddress]
      ,[YearlyIncome]
      ,[TotalChildren]
      ,[NumberChildrenAtHome]
      ,[EnglishEducation]
      ,[SpanishEducation]
      ,[FrenchEducation]
      ,[EnglishOccupation]
      ,[SpanishOccupation]
      ,[FrenchOccupation]
      ,[HouseOwnerFlag]
      ,[NumberCarsOwned]
      ,[AddressLine1]
      ,[AddressLine2]
      ,[Phone]
      ,[DateFirstPurchase]
      ,[CommuteDistance]
FROM   dbo.Stg_Customer AS s
UNION ALL  
-- Keep rows that are not being touched
SELECT [CustomerKey]
      ,[GeographyKey]
      ,[CustomerAlternateKey]
      ,[Title] 
      ,[FirstName]
      ,[MiddleName]
      ,[LastName]
      ,[NameStyle]
      ,[BirthDate]
      ,[MaritalStatus]
      ,[Suffix]
      ,[Gender]
      ,[EmailAddress]
      ,[YearlyIncome]
      ,[TotalChildren]
      ,[NumberChildrenAtHome]
      ,[EnglishEducation]
      ,[SpanishEducation]
      ,[FrenchEducation]
      ,[EnglishOccupation]
      ,[SpanishOccupation]
      ,[FrenchOccupation]
      ,[HouseOwnerFlag]
      ,[NumberCarsOwned]
      ,[AddressLine1]
      ,[AddressLine2]
      ,[Phone]
      ,[DateFirstPurchase]
      ,[CommuteDistance] 
	  FROM dbo.DimCustomer
	  AS p
WHERE NOT EXISTS
(   SELECT  *
    FROM    [dbo].[stg_Customer] s
    WHERE   s.[CustomerKey] = p.[CustomerKey]
)
;


(18484 rows affected)

Total execution time: 00:00:02.0165977

In [41]:
--Switch objects by renaming
RENAME OBJECT dbo.[DimCustomer]          TO [DimCustomer_old];
RENAME OBJECT dbo.[DimCustomer_upsert]  TO [DimCustomer];

--See that the data classifications are now on DimCustomer_old
SELECT
    sys.all_objects.name AS TableName, sys.all_columns.name As ColumnName,
    [Label], Label_ID, Information_Type, Information_Type_ID
FROM
          sys.sensitivity_classifications
left join sys.all_objects on sys.sensitivity_classifications.major_id = sys.all_objects.object_id
left join sys.all_columns on sys.sensitivity_classifications.major_id = sys.all_columns.object_id
                         and sys.sensitivity_classifications.minor_id = sys.all_columns.column_id;


(2 rows affected)

Total execution time: 00:00:00.3103288

TableName,ColumnName,Label,Label_ID,Information_Type,Information_Type_ID
DimCustomer_old,EmailAddress,Confidential,NULL,Contact Info,NULL
DimCustomer_old,Phone,Confidential,NULL,Contact Info,NULL


In [42]:
--Cleanup
DROP TABLE dbo.DimCustomer_old;
DROP TABLE dbo.Stg_Customer;

Commands completed successfully.

Total execution time: 00:00:00.4958549

Workaround: Instead of doing the rename object on your own, add the stored procedure below and use it. Re-run the first four sections above, down through populating the upsert table. Then call the stored procedure as shown in the bottom section

In [43]:
CREATE PROC SwapWithMetadata
    @SrcSchema NVARCHAR(128),
    @SrcTable NVARCHAR(128),
    @DestSchema NVARCHAR(128),
    @DestTable NVARCHAR(128),
    @TransferMetadata BIT,
    @DropOldTable BIT
AS

BEGIN
    SET NOCOUNT ON

    BEGIN TRY 
        --Check if destination table exists
        DECLARE @DestSchemaQualifiedTableName NVARCHAR(257)
        SET @DestSchemaQualifiedTableName = @DestSchema + '.' + @DestTable
        
        IF OBJECT_ID(@DestSchemaQualifiedTableName) IS NULL
        BEGIN
        DECLARE @DestErr NVARCHAR(MAX)
            SET @DestErr = 'Table ' + @DestSchemaQualifiedTableName + ' not found'
            RAISERROR(@DestErr, 15, 1)
        END

        --Check if source table exists
        DECLARE @SrcSchemaQualifiedTableName NVARCHAR(257)
        SET @SrcSchemaQualifiedTableName = @SrcSchema + '.' + @SrcTable

        IF OBJECT_ID(@SrcSchemaQualifiedTableName) IS NULL
            BEGIN
                DECLARE @SrcErr NVARCHAR(MAX)
                SET @SrcErr = 'Table ' + @SrcSchemaQualifiedTableName + ' not found'
                RAISERROR(@SrcErr, 15, 1)
            END
				
        --Move destination table to destination_old. Move source table to destination
        DECLARE @RenameSql NVARCHAR(MAX)
        SET @RenameSql = 'RENAME OBJECT ' + @DestSchemaQualifiedTableName + ' TO ' + @DestTable + '_old; '
        Set @RenameSql = @RenameSql + '  RENAME OBJECT ' + @SrcSchemaQualifiedTableName + ' TO ' + @DestTable
        PRINT 'Executing ' + @RenameSql + ' ...'
        EXEC sp_executesql @RenameSql;

        --drop temp table if it exists
        IF OBJECT_ID('tempDB..#tempApplySensitivityClassificationsToTable') IS NOT NULL
			DROP TABLE #tempApplySensitivityClassificationsToTable;

        --check if we should transfer data classifications from old to new table 
        IF ISNULL(@TransferMetadata,0) = 1 
		BEGIN

            --put current classifications in a temp table
            DECLARE @OldTable NVARCHAR(128) = @DestTable + '_old';
            WITH  CurrentClassifications as (
                SELECT
                    CAST('dbo' as NVARCHAR(128)) [Schema],
                    CAST(sys.all_objects.name as NVARCHAR(128)) [Table],
                    CAST(sys.all_columns.name as NVARCHAR(128)) [Column],
                    CAST([Information_Type] as NVARCHAR(128)) [Informationtype],
                    CAST([Label] as NVARCHAR(128)) [Label]
                FROM
                    sys.sensitivity_classifications
                    LEFT OUTER JOIN sys.all_objects on sys.sensitivity_classifications.major_id = sys.all_objects.object_id
                    LEFT OUTER JOIN sys.all_columns on sys.sensitivity_classifications.major_id = sys.all_columns.object_id
                        and sys.sensitivity_classifications.minor_id = sys.all_columns.column_id

            )
            SELECT ROW_NUMBER() OVER (ORDER BY [Schema], [Table], [Column]) [ID],
                [Schema], [Table], [Column], [Informationtype], [Label]
            INTO #tempApplySensitivityClassificationsToTable
            FROM CurrentClassifications
            WHERE [Schema] = @DestSchema AND [Table] = @OldTable;


            DECLARE @i INT
            SET @i = 1
            DECLARE @Max INT
            SELECT @Max = COUNT(*)
            FROM #tempApplySensitivityClassificationsToTable;

            PRINT 'Transferring ' + CAST(@Max as VARCHAR(4)) + ' classifications'
            --drop and recreate sensitivity classifications
            DECLARE @Sql NVARCHAR(MAX)
            DECLARE @Col NVARCHAR(128)
            DECLARE @InfoType NVARCHAR(128)
            DECLARE @Label NVARCHAR(128)

        WHILE @i <= @Max
			BEGIN
                SELECT @Col = [Column], @InfoType = [InformationType], @Label = [Label]
                FROM #tempApplySensitivityClassificationsToTable
                WHERE Id = @i

                SET @Sql = 'DROP SENSITIVITY CLASSIFICATION FROM ' + @DestSchemaQualifiedTableName + '.' + @Col
                PRINT 'Executing ' + @Sql + '...'
                EXEC sp_executesql @Sql

                SET @Sql = 'ADD SENSITIVITY CLASSIFICATION TO ' + @DestSchemaQualifiedTableName + '.' + @Col
                IF (@InfoType IS NOT NULL AND @Label IS NOT NULL)
                    BEGIN
                        SET @Sql = @Sql + ' WITH (LABEL=''' + @Label + ''', INFORMATION_TYPE=''' + @InfoType + ''')'
                    END
                ELSE IF (@InfoType IS NOT NULL)
                    BEGIN
                        SET @Sql = @Sql + ' WITH (INFORMATION_TYPE=''' + @InfoType + ''')'
                    END
                ELSE IF (@Label IS NOT NULL)
                    BEGIN
                        SET @Sql = @Sql + ' WITH (LABEL=''' + @InfoType + ''')'
                    END
                ELSE
                    BEGIN
                        SET @Sql = NULL
                    END
                IF (@Sql IS NOT NULL)
                    BEGIN
                        PRINT 'Executing ' + @Sql + '...'
                        EXEC sp_executesql @Sql
                    END

                SET @i = @i + 1
            END
        END

        IF ISNULL(@DropOldTable,0) = 1 
        BEGIN
            DECLARE @DropSql NVARCHAR(MAX)
            SET @DropSql = 'DROP TABLE ' + @DestSchemaQualifiedTableName + '_old;'
            PRINT 'Executing ' + @DropSql + '...'
            EXEC sp_executesql @DropSql;
        END

	END TRY 
	
    BEGIN CATCH 
	    Print 'ERROR... Procedure: ' + ERROR_PROCEDURE() + '  Message: ' + ERROR_MESSAGE()
	END CATCH

END
 

: Msg 2714, Level 16, State 3, Line 1
There is already an object named 'SwapWithMetadata' in the database.

Total execution time: 00:00:00.1032211

In [54]:
EXEC SwapWithMetadata 'dbo', 'DimCustomer_Upsert', 'dbo','DimCustomer',1,1

Executing RENAME OBJECT dbo.DimCustomer TO DimCustomer_old; RENAME OBJECT dbo.DimCustomer_Upsert TO DimCustomer ...

Transferring 2 classifications

Executing DROP SENSITIVITY CLASSIFICATION FROM dbo.DimCustomer.EmailAddress...

Executing ADD SENSITIVITY CLASSIFICATION TO dbo.DimCustomer.EmailAddress WITH (LABEL='Confidential', INFORMATION_TYPE='Contact Info')...

Executing DROP SENSITIVITY CLASSIFICATION FROM dbo.DimCustomer.Phone...

Executing ADD SENSITIVITY CLASSIFICATION TO dbo.DimCustomer.Phone WITH (LABEL='Confidential', INFORMATION_TYPE='Contact Info')...

Executing DROP TABLE dbo.DimCustomer_old;...

Total execution time: 00:00:03.3317753

In [56]:
--Cleanup
Drop table dbo.Stg_Customer;

Commands completed successfully.

Total execution time: 00:00:00.2659364